<strong>Date :</strong> Créé le 12 Février 2021| Mis à jour le 24 Février 2021 </strong>

<strong>Compétition Kaggle - Team Théo
    
@auteur : </strong>Théo SACCAREAU & Théo VEDIS

<strong>(1)_observation_cleaning
      
Description :</strong> Dans ce premier Notebook, nous prendrons connaissance des données à disposition (observation) puis apporterons les premières modifications (nettoyage). 

Temps d'exécution du Notebook : environ 8 min (2min sans le téléchargement depuis l'API de Kaggle). 


# Importation des librairies

In [ ]:
import pandas as pd
import numpy as np
import random 
import json 

# Téléchargement des données

In [ ]:
# Chemin relatif vers le dossier "data" (inutile de le changer).
pathFile = "../data/" 

## Accès à l'API Kaggle avec Jupyter Notebook (via Linux)

<strong>(!) Partie à ignorer si le fichier "comments_students.csv" est déjà téléchargé et placé dans le dossier "data". (!)</strong>

In [ ]:
# Création du dossier ".kaggle" qui devra contenir le fichier "kaggle.json"
# téléchargé sur le site de Kaggle (cf étape suivante). 
!mkdir ~/.kaggle/ 

<strong>AVANT</strong> d'effectuer la ligne qui suit il est important de suivre les étapes suivantes (comme indiqué dans le README.txt): 
- Connectez-vous à votre compte Kaggle 
- Cliquez sur votre profil et rendez-vous à la rubrique "Account" 
- Dans la zone "API", cliquez sur "Create New API Token". 
- Enregistrez le fichier "kaggle.json" (par Défaut dans vos paramètres). 

In [ ]:
# On copie le fichier "kaggle.json" qui se trouve dans le dossier  
# "Téléchargements" dans le dosier ".kaggle" crée juste au-dessus. 

# (!) Il est possible que le chemin ne corresponde pas tout à fait au votre
# dans ce cas effectuez les changements nécessaires. (!) 
!cp ~/Téléchargements/kaggle.json ~/.kaggle

In [ ]:
# Le résultat de cette commande doit afficher : "kaggle.json" (valide que le
# fichier a bien été copié dans le répertoire).
!cd ~/.kaggle && ls

In [ ]:
# Téléchargement du fichier .zip de la compétition 
!kaggle competitions download -c 2021-reddit-score-prediction

In [ ]:
# On dézipe le fichier 
# (!) Si unzip n'est pas installe il faut exécuter également le "sudo apt-get"
#!sudo apt-get install unzip
!unzip 2021-reddit-score-prediction.zip 

In [ ]:
# Enfin on déplace le ficher "comments_students.csv" dans le dossier "data".
!mv comments_students.csv data/comments_students.csv 

<strong>(!) Fin de la partie à ignorer (!)</strong>

## Chargement des données 

In [ ]:
# Téléchargement des données pour la compétition.
# Temps de chargement : 30s.
df = pd.read_csv(pathFile + "comments_students.csv") 

In [ ]:
# Aperçu pour vérifier que le chargement a bien été effectué
df.head()

,created_utc,ups,subreddit_id,link_id,name,subreddit,id,author,body,parent_id
0,1430438400,3.0,t5_2qh1i,t3_34f9rh,t1_cqug90j,AskReddit,cqug90j,jesse9o3,No one has a European accent either because i...,t1_cqug2sr
1,1430438400,3.0,t5_2qh1i,t3_34fvry,t1_cqug90k,AskReddit,cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry
2,1430438400,5.0,t5_2qh1i,t3_34ffo5,t1_cqug90z,AskReddit,cqug90z,InterimFatGuy,NSFL,t1_cqu80zb
3,1430438401,1.0,t5_2qh1i,t3_34aqsn,t1_cqug91c,AskReddit,cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m
4,1430438401,101.0,t5_2qh1i,t3_34f9rh,t1_cqug91e,AskReddit,cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc


# (1) Observation des données 
Avant toutes manipulations des données, il est nécessaire de bien comprendre le contenu du jeu de données. 

In [ ]:
print("Nombre de colonnes/variables : %i \nNombre de lignes : %i" \
      %(df.shape[1], df.shape[0])) 

Nombre de colonnes/variables : 10 
Nombre de lignes : 4234970


Avec ce jeu de données, nous disposons de 10 variables et de plus de 4,2 millions de commentaires Reddit. Observons leur type.

In [ ]:
# Informations sur les types de variables
print(df.info()) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4234970 entries, 0 to 4234969
Data columns (total 10 columns):
 #   Column        Dtype  
---  ------        -----  
 0   created_utc   int64  
 1   ups           float64
 2   subreddit_id  object 
 3   link_id       object 
 4   name          object 
 5   subreddit     object 
 6   id            object 
 7   author        object 
 8   body          object 
 9   parent_id     object 
dtypes: float64(1), int64(1), object(8)
memory usage: 323.1+ MB
None


Parmi les 10 variables, 8 sont des variables qualitatives (chaine de caractères) et 2 quantitatives (dont une avec des valeurs entières `int` et l'autre des valeurs décimales `float`). 

Pour poursuivre notre observation, il peut être intéressant de tirer aléatoirement 10 commentaires, et d'observer le contenu des variables. Cela nous permettra de détailler le contenu de chacune des variables. 

In [ ]:
# On indique à random la "graine" pour avoir un résultat identique à chaque 
# exécution (et donc des analyses qui correspondent aux résultats obtenus).
random.seed(10)

# On récupère 10 indexes aléatoirement parmi la liste des indexes du dataframe.
index = random.sample(list(df.index), 10)

# Et on affiche la partie du DataFrame correspondante.
df.iloc[index]

,created_utc,ups,subreddit_id,link_id,name,subreddit,id,author,body,parent_id
273343,1430625233,1.0,t5_2qh1i,t3_34nl4h,t1_cqwnxuc,AskReddit,cqwnxuc,IamtheRadar,"I don't even care what they think, it's disapp...",t1_cqwh0l4
3597782,1432751839,NaN,t5_2qh1i,t3_37gspl,t1_crmr8wh,AskReddit,crmr8wh,Not_Prisoner,I used to work at a theater back in hs. Our t...,t1_crml6bw
4048020,1432981048,NaN,t5_2qh1i,t3_37t5nq,t1_crps56a,AskReddit,crps56a,kane55,I grew up and went to high school in the 1980'...,t3_37t5nq
124423,1430514545,1.0,t5_2qh1i,t3_34huks,t1_cqvf80o,AskReddit,cqvf80o,pneighthaughn,I can throw most any knife and stick it in a t...,t3_34huks
1728877,1431552671,-1.0,t5_2qh1i,t3_35vj7q,t1_cr87esl,AskReddit,cr87esl,stas02,Moving to Somalia,t3_35vj7q
3880406,1432895575,NaN,t5_2qh1i,t3_37paiu,t1_cronyrp,AskReddit,cronyrp,Caaluminum,What's your aphrodisiac?,t1_cronutd
4121293,1433039433,NaN,t5_2qh1i,t3_37vv9d,t1_crqgrgv,AskReddit,crqgrgv,[deleted],[deleted],t3_37vv9d
2328012,1431937255,2.0,t5_2qh1i,t3_36ci3t,t1_crcrhvr,AskReddit,crcrhvr,GloboX38,Cyanide capsule. Two in one.,t3_36ci3t
1344102,1431321021,2.0,t5_2qh1i,t3_35jfjt,t1_cr587zj,AskReddit,cr587zj,DoinItDirty,Yeah man that happens,t1_cr584wl
288598,1430638715,1.0,t5_2qh1i,t3_34oc3u,t1_cqwrywm,AskReddit,cqwrywm,Trezzie,Good man. That issue is really painful. I did ...,t1_cqwrt7o


Ce jeu de données contient un commentaire par ligne. Chaque commentaire est décrit avec les 10 attributs suivants :

* La variable `created_utc` qui permet de connaitre le moment de publication du commentaire est au format 'UTC'. C'est une échelle de temps adoptée comme base du temps civil international par la majorité des pays du globe. Elle est comprise entre le temps atomique international (TAI) qui est stable mais déconnecté de la rotation de la Terre et le temps universel (TU), directement lié à la rotation de la Terre. <br> Concrétement, les données dans cette colonne correspond au nombre de secondes écoulées depuis le 1er janvier 1970 00:00:00. Aucune de ces valeurs ne peut être nulle. 

* `ups` est le score d'un commentaire, c'est-à-dire la somme de likes (+1) ou dislikes (-1) qu'il a générée. Les valeurs 'NaN' qu'on aperçoit signifie que ces commentaires constitue la base de test. C'est donc bien évidemment ce qu'on doit prédire. 

* `subreddit_id` correspond à l'identifiant du "sous-reddit", c'est-à-dire une sous-partie du site consacrée à un thème spécifique. On constate qu'il est indique pour les 10 commentaires tirés aléatoirement. On vérifiera si cette tendance est valable pour l'ensemble des commentaire, ou s'il y a des exceptions. 

* `link_id` permet de connaitre l'identifiant du lien dans lequel se trouve le commentaire. En d'autres termes, il permet de connaitre le post (le sujet / le topic) auquel le commentaire répond, soit une réponse direct au post, soit une réponse à un commentaire. 

* `name` correspond au nom complet du commentaire.

* `subreddit` c'est l'équivalent de la colonne `subreddit_id` sauf qu'au lieu d'avoir un identifiant, on a ici le nom complet, à savoir ici "AskReddit". Comme pour `subreddit_id`, il faudra vérifier si les valeurs sont identiques pour chaque commentaire. Si c'est effectivement le cas, cette colonne n'apporte donc aucune information. 

* `id` correspond à l'identifiant du commentaire. Cet attribut est strictement équivalent à `name`, sauf qu'il ne comporte pas le type 't1_' en préfixe. 

* `author` correspond au pseudo de la personne qui a écrit le commentaire. On s'aperçoit que certain compte on été supprimé (valeur `[deleted]`). 

* `body` correspond au contenu du commentaire. Comme pour l'auteur, ce contenu peut avoir été supprimé. 

* Enfin, la colonne `parent_id`, nous permet de connaitre l'identifiant de l'élement auquel le commentaire répond. Ca peut être une réponse à un post (dans ce cas là l'identifiant sera précédé de 't3_') ou une réponse à un commentaire le préfixe sera alors 't1_'). 
<br>
Désormais, nous pouvons vérifier que notre supposition concernant les types d'éléments (t1=commentaire, t3=post/sujet/topic et t5=subreddit) est justifiée :

In [ ]:
list_col = ['name', 'link_id', 'subreddit_id']
list_elem = ['comment', 'post', 'subreddit']
for column, elem in zip(list_col, list_elem):
    print("List of unique prefixes for %s  = % s" %
          (elem, df[column].apply(lambda x: x[:2]).unique()))

List of unique prefixes for comment  = ['t1']
List of unique prefixes for post  = ['t3']
List of unique prefixes for subreddit  = ['t5']


Notre intuition était donc la bonne. Nous pouvons connaitre le type d'un élément grâce à son préfixe. 
<br> <br>
Pour finir, analysons un tableau renseignant des informations plus précises sur chaque variable : la moyenne (pour les variables numériques), la fréquence, l'élément qui apparait le plus souvent etc. 

In [ ]:
# Tableau récapitulatif 
df.iloc[:,range(0,10)].describe(include='all')

,created_utc,ups,subreddit_id,link_id,name,subreddit,id,author,body,parent_id
count,4.234970e+06,3.218512e+06,4234970,4234970,4234970,4234970,4234970,4234970,4234909,4234970
unique,NaN,NaN,1,148848,4234970,1,4234970,570735,3675455,1464558
top,NaN,NaN,t5_2qh1i,t3_37c2p3,t1_cr5jarg,AskReddit,crhb55q,[deleted],[deleted],t3_37pr7d
freq,NaN,NaN,4234970,35812,1,4234970,1,312007,284241,30771
mean,1.431798e+09,1.274048e+01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,7.720695e+05,1.257486e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.430438e+09,-3.330000e+02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.431104e+09,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.431802e+09,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.432483e+09,2.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* La ligne `count` nous permet de connaitre le nombre de ligne non-nulles pour chaque variable. On constate que seules deux variables ont des valeurs nulles : `ups` et `body`. Pour la première, ce n'est pas une surprise puisqu'on sait qu'une partie des commentaires est réservée à l'échantillon "test" et n'a donc pas de score. En revanche, pour la variable `body` c'est plus contrayant. En effet, les valeurs nulles peuvent empecher les algorithmes de machine learning de fonctionner corrrectement. Il faudra modifier ces valeurs en les remplaçant par "[deleted]". 

* `unique` nous permet de savoir le nombre d'élements distincts que comporte chaque variable. On a donc la confirmation que les variables `subreddit_id` et `subreddit` ont la même valeur pour tous les commentaires, ce sont donc deux colonnes <strong> inutiles </strong>.  <br>
On apprend également qu'il y a 148 900 sujets/posts et 570 000 auteurs différents. <br> 
En revanche, les identifiants et noms de commenatires sont bien uniques. 

* Les deux lignes suivantes (`top` et `freq`), nous permet de connaitre l'élément qui apparait le plus souvent pour une variable, et son nombre d'apparition. <br> 
On constate par exemple que le sujet le plus commenté (directement ou indirectement) porte l'identifiant "t3_37c2p3", il a reçu près de 36 000 commentaires. <br>
Une information qu'il ne faut pas confondre avec celle de la colonne `parent_id`. Effectivement, avec celle-ci on constate que le commentaire qui a reçu le plus de commentaires DIRECTEMENT (sans compter les commentaires de commentaires) est le 't3_37pr7d' avec plus de 30 000 commentaires directs. <br> 
On constate également que pour 312 000 commentaires, l'auteur a été supprimé. Pour 284 000 commentaires, c'est le body qui n'existe plus. 

* Enfin, pour les variables numériques, on peut remarquer que le score (`ups`) maximal est de 6761 alors que le plus bas est de -333. En moyenne le score est de 12,7. A noter qu'au moins 50% des scores sont inférieurs ou égaux à 1. <br> 
Concernant l'heure, tous les commentaires ont été écrits entre 1430438400 (soit le 1er Mai 2015 à 0h 00min 00s) et 1433116799 (soit le 31 Mai 2015 à 23h 59min 59s). 


<strong>BILAN : </strong><br>
Après avoir observé et analysé les données mises à notre disposions, voici les principaux enseignants que nous avons pû faire.<br> 
- Tous les commentaires proviennent de la même sous-partie du site 'Reddit', à savoir la partie "AskReddit". Dans cette partie, les utilisateurs peuvent soumettre des questions ouvertes auxquelles d'autres utilisateurs peuvent ensuite répondre. En juillet 2015, AskReddit était le sous-reddit le plus populaire de tout le Reddit et en février 2020, il comptait 26 millions de membres. Ainsi l'information apportée par les variables "subreddit_id" et "subreddit" est identique pour chaque article. Elles ne nous permettront pas de prédire le score d'un commentaire. Il faudra donc les supprimer lors de l'étape suivante.
<br> 
- Ensuite, nous avons compris la différence entre les 3 préfixes 't1', 't3' et 't5'. Ils permettent de connaitre à quel élément du site l'identifiant correspond. En effet, un identifiant de commentaire aura pour préfixe 't1', un post/sujet aura le préfixe 't3' et le subreddit 't5'. 
<br> 
- Ainsi, pour chaque commentaire nous aurons : son identifiant et son nom (nous n'en garderons que le nom car l'identifiant n'est rien d'autre que le nom sans le préfixe 't1'), l'identifiant de son parent (post 't3' ou autre commentaire 't1'), et l'identifiant du post auquel il répond directement ou pas. Sans oublier, son heure de publication, son auteur, son contenu et son socre (uniquement pour l'échantillon d'apprentissage). 

# (2) Nettoyage des données
Après avoir observé attentivement les données, nous pouvons passer à une première phase de nettoyage. <br>
En Machine Learning, on utilise généralement la méthode dites « des 4C » (pour ‘Correcting’, ‘Completing’, ‘Creating’ et ‘Converting’) pour nettoyer le jeu de données.
- 'Correcting' = modifier ou supprimer les valeurs incohérentes.
- 'Completing' = remplacer les valeurs nulles (les valeurs manquantes)
- 'Creating' = utiliser des variables existantes pour en créer de nouvelles
- 'Converting' = formater les types des variables
<br>
<br> 

### 2-1 Correcting 
Suite à notre observation des données dans la première partie, nous avons pu détecter que certaines variables pouvaient être supprimées car leurs informations n'apportent aucune distinction entre les commentaires (`subreddit` et `subreddit_id`) ou une information redondante (`id`). C'est lors de la phase 'Correcting' que nous supprimons ou modifions les valeurs inutiles ou incohérentes. 

In [ ]:
# Colonne à supprimer
column_to_delete  = ["subreddit_id", "subreddit", "id"]

# Suppressions des colonnes 
df = df.drop(columns=column_to_delete)

# Vérification 
print(df.columns)

Index(['created_utc', 'ups', 'link_id', 'name', 'author', 'body', 'parent_id'], dtype='object')


Dans l'optique du traitement de texte que nous effectuerons prochainement, nous pensons qu'il est plus utile de remplacer le contenu des commentaires supprimés (`[deleted]`) par une chaîne vide. 

In [ ]:
# On chaque contenu du commentaire qui vaut '[deleted]' on le remplace par
# une chaîne vide.
df['body'] = df['body'].apply(
    lambda x: '' if x is not np.nan and x == '[deleted]' else x)

### 2-2 Completing
L’étape ‘Completing’ a pour but de remplacer les valeurs nulles (les valeurs manquantes). Cette étape est nécessaire car certains algorithmes de Machine Learning ne savent pas comment traiter ces valeurs et échoueront. 
<br><br>  
Pour rappel, nous avons vu juste au-dessus que seuls deux attributs avaient des valeurs nulles : `ups` et `body`. Pour `ups` ce n'est pas une anomalie, puisque les valeurs nulles permettent d'identifier les commentaires constituant l'échantillon de test. En revanche les valeurs nulles pour la variable `body` sont plus à risque et ils faut les modifier. On choisi simplement de les remplacer par une chaîne vide. 

In [ ]:
print("Number of null values BEFORE the completing step : ",
      np.sum(df['body'].isnull()))

# On remplace les valeurs nulles par une chaîne vide
df['body'] = df['body'].fillna('')

print("Number of null values AFTER the completing step : ",
      np.sum(df['body'].isnull()))

Number of null values BEFORE the completing step :  61
Number of null values AFTER the completing step :  0


### 2-3 Créating
L'objectif de cette étape est d'utiliser des variables existantes pour en créer de nouvelles. <br> <br> 
Même si la définition et l'implémentation de nouvelles features constituera une étape essentielle du projet par la suite, nous pouvons dès à présent créer de nouvelles variables à partir des variables existantes. <br> 
Par exemple, pour ce qui concerne le parent d'un commenataire, il peut-être intéressant de séparer son identifiant et son type. En effet, garder l'identifiant permettra de savoir s'il répond à un commentaire ou post populaire (avec un score élevé donc). De plus, avoir une variable permettant de savoir si le parent est directement un post ou juste un commentaire, peut être un indicateur pour prédire le score. Certes, ces deux informations sont actuellement regroupées dans le même variable, mais nous jugeons plus pertinent (et espérons que cela aidera l'algorithme à mieux prédire) de séparer ces deux informations en deux variables distinctes. 

In [ ]:
# On crée une nouvelle variable pour le type du parent. 
# On garde donc les deux premiers caractères de la variable 'parent_id'.
df['parent_type'] = df['parent_id'].apply(lambda x : x[:2])

# Vérification : On est censé avoir deux valeurs : 't1' (pour les commentaires) 
# ou 't3' (pour les posts).
print(df['parent_type'].unique())

['t1' 't3']


On crée également deux variables binaires permettant de savoir si l'auteur et/ou le contenu d'un commentaire a été supprimé ou non. 

In [ ]:
# On donne la valeur 1 si l'auteur a été supprimé ('[deleted]'), 0 sinon.
df['is_author_deleted'] = df['author'].apply(
    lambda x: 1 if x == '[deleted]' else 0)

# Vérification : On est censé avoir deux valeurs : 1 et 0.
print(df['is_author_deleted'].unique())

[0 1]


In [ ]:
# Même chose pour le body.
df['is_body_deleted'] = df['body'].apply(lambda x: 1 if x == '' else 0)

# Vérification : On est censé avoir deux valeurs : 1 et 0.
print(df['is_body_deleted'].unique())

[0 1]


### 2-4 Converting 
Enfin, la quatrième et ultime étape consiste à formater les types des variables. 

Dans notre exemple, il peut être plus pertinent de transformer la variable `parent_type` en variable binaire (plutôt que de garder une variable qualitative). On renomme donc cette variable `parentIsComment`, elle vaudra 1 quand le parent du commentaire est lui aussi un commentaire, 0 sinon (si c'est un post). 


In [ ]:
# Conversion 
df['parent_type'] = df['parent_type'].apply(lambda x: 1 if x == 't1' else 0)

# Renommage
df = df.rename(columns={'parent_type': 'is_parent_comment'})

# Vérification 
df['is_parent_comment'].head()

0    1
1    0
2    1
3    1
4    1
Name: is_parent_comment, dtype: int64

# Bilan & Sauvegarde 
On observe le contenu des premières lignes du DataFrame modifié, et on le sauvegarde dans un fichier pour qu'il puisse être réutiliser dans les Notebooks suivants (qui auront pour but de créer les features). 

In [ ]:
# Aperçu avant sauvegarde
df.head()

,created_utc,ups,link_id,name,author,body,parent_id,is_parent_comment,is_author_deleted,is_body_deleted
0,1430438400,3.0,t3_34f9rh,t1_cqug90j,jesse9o3,No one has a European accent either because i...,t1_cqug2sr,1,0,0
1,1430438400,3.0,t3_34fvry,t1_cqug90k,beltfedshooter,That the kid ..reminds me of Kevin. so sad :-(,t3_34fvry,0,0,0
2,1430438400,5.0,t3_34ffo5,t1_cqug90z,InterimFatGuy,NSFL,t1_cqu80zb,1,0,0
3,1430438401,1.0,t3_34aqsn,t1_cqug91c,JuanTutrego,I'm a guy and I had no idea this was a thing g...,t1_cqtdj4m,1,0,0
4,1430438401,101.0,t3_34f9rh,t1_cqug91e,dcblackbelt,"Mid twenties male rocking skinny jeans/pants, ...",t1_cquc4rc,1,0,0


In [ ]:
# Sauvegarde DataFrame dans un fichier 
df.to_json(pathFile + "df_clean.json")